BL.EN.U4CSE21176 ; S NAVIN SUNDER ; CSE - C

In [25]:
import pandas as pd
from collections import defaultdict
from scipy.stats import chi2_contingency
from itertools import product   #used to iterate over the dataset 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB  #gaussianNB uses numeric input. Ordinal encoding is used in this case to get numeric input
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

A1. For the data provided below, calculate the prior probability for each class.

In [ ]:
#prior probability = probability of an event before it is collected


In [6]:
dataset = [
    {'age': '<=30', 'income': 'high', 'student': 'no', 'credit_rating': 'fair', 'buys_computer': 'no'},
    {'age': '<=30', 'income': 'high', 'student': 'no', 'credit_rating': 'excellent', 'buys_computer': 'no'},
    {'age': '31..40', 'income': 'high', 'student': 'no', 'credit_rating': 'fair', 'buys_computer': 'yes'},
    {'age': '>40', 'income': 'medium', 'student': 'no', 'credit_rating': 'fair', 'buys_computer': 'yes'},
    {'age': '>40', 'income': 'low', 'student': 'yes', 'credit_rating': 'fair', 'buys_computer': 'yes'},
    {'age': '>40', 'income': 'low', 'student': 'yes', 'credit_rating': 'excellent', 'buys_computer': 'no'},
    {'age': '31..40', 'income': 'low', 'student': 'yes', 'credit_rating': 'excellent', 'buys_computer': 'yes'},
    {'age': '<=30', 'income': 'medium', 'student': 'no', 'credit_rating': 'fair', 'buys_computer': 'no'},
    {'age': '<=30', 'income': 'low', 'student': 'yes', 'credit_rating': 'fair', 'buys_computer': 'yes'},
    {'age': '>40', 'income': 'medium', 'student': 'yes', 'credit_rating': 'fair', 'buys_computer': 'yes'},
    {'age': '<=30', 'income': 'medium', 'student': 'yes', 'credit_rating': 'excellent', 'buys_computer': 'yes'},
    {'age': '31..40', 'income': 'high', 'student': 'no', 'credit_rating': 'excellent', 'buys_computer': 'yes'},
    {'age': '31..40', 'income': 'medium', 'student': 'yes', 'credit_rating': 'fair', 'buys_computer': 'no'},
    {'age': '>40', 'income': 'high', 'student': 'no', 'credit_rating': 'excellent', 'buys_computer': 'no'}
]

# Calculate the total number of instances
totalInstances = len(dataset)

# Calculate the prior probability for each class
classes = set(entry['buys_computer'] for entry in dataset)
priorProbabilities = {}

for class_label in classes:
    class_count = sum(1 for entry in dataset if entry['buys_computer'] == class_label)
    priorProbabilities[class_label] = class_count / totalInstances

print("Prior Probabilities:")
for class_label, prior_probability in priorProbabilities.items():
    print(f"P({class_label}) = {prior_probability:.4f}")


Prior Probabilities:
P(no) = 0.4286
P(yes) = 0.5714


A2.Calculate the class conditional densities for various features & classes. Observe if any class conditional density has zero values. 

In [7]:
# Initialize dictionaries to store class conditional densities
class_conditional_densities = defaultdict(dict) #defaultdict is a sub-class of a dictonary class that returns a dictionary like objects

# Calculate class conditional densities for each feature and class
for feature in dataset[0].keys():  # Assuming features are in the first sublist
    if feature != 'buys_computer':  # Exclude the class label
        for class_label in classes:
            # Filter instances with the given feature and class
            feature_class_instances = [entry for entry in dataset if entry['buys_computer'] == class_label and entry[feature] == entry[feature]]
            
            # Calculate class conditional density
            density = len(feature_class_instances) / priorProbabilities[class_label]
            class_conditional_densities[class_label][feature] = density

# Print the class conditional densities
print("Class Conditional Densities:")
for class_label, densities in class_conditional_densities.items():
    print(f"For class {class_label}:")
    for feature, density in densities.items():
        print(f"P({feature} | {class_label}) = {density:.4f}")

# Check if any class conditional density has zero values
zero_density_features = [(class_label, feature) for class_label, densities in class_conditional_densities.items() for feature, density in densities.items() if density == 0]

if zero_density_features:
    print("\nFeatures with zero class conditional densities:")
    for class_label, feature in zero_density_features:
        print(f"For class {class_label}, feature {feature} has zero density.")
else:
    print("\nNo features have zero class conditional densities.")


Class Conditional Densities:
For class no:
P(age | no) = 14.0000
P(income | no) = 14.0000
P(student | no) = 14.0000
P(credit_rating | no) = 14.0000
For class yes:
P(age | yes) = 14.0000
P(income | yes) = 14.0000
P(student | yes) = 14.0000
P(credit_rating | yes) = 14.0000

No features have zero class conditional densities.


A3. Test for independence between the 4 given features. 

In [13]:
#chi2_contingency null hypotheses => to check the relationship between the two variables that determines if the features are independent or not 

# Extract features and class labels from the dataset
features = [entry['age'] for entry in dataset]
income = [entry['income'] for entry in dataset]
student = [entry['student'] for entry in dataset]
credit_rating = [entry['credit_rating'] for entry in dataset]
buys_computer = [entry['buys_computer'] for entry in dataset]

# Create a list of unique values for each feature
unique_values = {
    'age': set(features),
    'income': set(income),
    'student': set(student),
    'credit_rating': set(credit_rating),
    'buys_computer': set(buys_computer)
}

# Generate all possible combinations of values for the features
combinations = list(product(unique_values['age'], unique_values['income'], unique_values['student'], unique_values['credit_rating'], unique_values['buys_computer']))

# Create a contingency table with Laplace smoothing
contingency_table = []
epsilon = 1e-10  # small constant to avoid zero expected frequencies

for age, inc, stud, credit, buys in combinations:
    count = sum(1 for i in range(totalInstances) if features[i] == age and buys_computer[i] == buys and income[i] == inc and student[i] == stud and credit_rating[i] == credit)
    contingency_table.append([count + epsilon])

# Perform Chi-squared test for independence
chi2, p, _, _ = chi2_contingency(contingency_table)

# Print the test results
print(f"Chi-squared value: {chi2}")
print(f"P-value: {p}")

# Check the significance level (e.g., alpha = 0.05)
alpha = 0.05    #alpha can be set to any value between 0 and 1
print("\nTest Result:")
if p < alpha:
    print("Reject the null hypothesis. Features are not independent.")
else:
    print("Fail to reject the null hypothesis. Features are independent.")


Chi-squared value: 0.0
P-value: 1.0

Test Result:
Fail to reject the null hypothesis. Features are independent.


A4. Build a Naïve-Bayes (NB) classifier for the above given data.

In [21]:
# Extract the features and target variable
features = np.array([[row['age'], row['income'], row['student'], row['credit_rating']] for row in dataset])
target = np.array([row['buys_computer'] for row in dataset])

# Convert categorical data to numerical values using LabelEncoder
label_encoder = LabelEncoder()
for i in range(features.shape[1]):
    features[:, i] = label_encoder.fit_transform(features[:, i])

# Convert features to numeric values
features = features.astype(float)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the Gaussian Naive Bayes classifier
gnb_classifier = GaussianNB()
gnb_classifier.fit(X_train, y_train)

# Make predictions on the test set
predictions = gnb_classifier.predict(X_test)

# Evaluate the classifier
accuracy = np.sum(predictions == y_test) / len(y_test)
print(f'Accuracy: {accuracy:.2%}')


Accuracy: 66.67%


A5. Build a NB classifier for your own project data. 

In [26]:
# Read data from Excel
data = pd.read_excel(r"C:\Amrita cse\5th Semester\Machine Learning\Lab\Lab 4\embeddingsdata.xlsx")

# Separate features and target
features = data.iloc[:, :-1]  # Features
target = data.iloc[:, -1]      # Target data 

# Convert categorical data to numerical values using LabelEncoder
label_encoder = LabelEncoder()
for col in features.columns:
    features[col] = label_encoder.fit_transform(features[col])

# Convert features to numeric values
features = features.astype(float)

# Split the data into training and testing sets
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create and train the Gaussian Naive Bayes classifier
gnb_classifier = GaussianNB()
gnb_classifier.fit(features_train, target_train)

# Make predictions on the test set
predictions = gnb_classifier.predict(features_test)

# Evaluate the classifier
accuracy = np.sum(predictions == target_test) / len(target_test)
print(f'Accuracy: {accuracy:.2%}')


Accuracy: 55.00%
